In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/danieloliveiradebrito/Projetos/fakenews


/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Load dataset

In [9]:
import datasets
from sklearn.model_selection import train_test_split

from fakenews.read_data import read_fake_recogna

df = read_fake_recogna().to_pandas()

train_full, test = train_test_split(df,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=df["label"]
                            )

train, val = train_test_split(train_full,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=train_full["label"]
                            )

ds = datasets.DatasetDict()
ds["train"] = datasets.Dataset.from_pandas(train.reset_index(drop=True))
ds["test"] = datasets.Dataset.from_pandas(test.reset_index(drop=True))
ds["validation"] = datasets.Dataset.from_pandas(val.reset_index(drop=True))


## BERT

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("adalbertojunior/distilbert-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("adalbertojunior/distilbert-portuguese-cased", do_lower_case=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
from fakenews.train import FakeNewsTrainer

fake_recogna_trainer = FakeNewsTrainer(ds)
tokenized_ds = fake_recogna_trainer.tokenize_ds()
trainer = fake_recogna_trainer.get_trainer(tokenized_ds)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1905/1905 [00:00<00:00, 2416.23 examples/s]


In [27]:
trainer.train()

 18%|█▊        | 500/2856 [04:36<21:16,  1.85it/s]

{'loss': 0.3086, 'grad_norm': 0.08902580291032791, 'learning_rate': 1.649859943977591e-05, 'epoch': 0.53}


                                                  
 33%|███▎      | 952/2856 [09:22<17:15,  1.84it/s]

{'eval_loss': 0.19316671788692474, 'eval_accuracy': 0.946981627296588, 'eval_runtime': 39.4993, 'eval_samples_per_second': 48.229, 'eval_steps_per_second': 6.051, 'epoch': 1.0}


 35%|███▌      | 1000/2856 [09:52<17:01,  1.82it/s] 

...

                                                   
100%|██████████| 2856/2856 [28:43<00:00,  1.85it/s]

{'eval_loss': 0.2224578559398651, 'eval_accuracy': 0.958005249343832, 'eval_runtime': 35.375, 'eval_samples_per_second': 53.852, 'eval_steps_per_second': 6.756, 'epoch': 3.0}


100%|██████████| 2856/2856 [28:45<00:00,  1.66it/s]

{'train_runtime': 1725.6626, 'train_samples_per_second': 13.24, 'train_steps_per_second': 1.655, 'train_loss': 0.14849792608693868, 'epoch': 3.0}


TrainOutput(global_step=2856, training_loss=0.14849792608693868, metrics={'train_runtime': 1725.6626, 'train_samples_per_second': 13.24, 'train_steps_per_second': 1.655, 'total_flos': 3026615124492288.0, 'train_loss': 0.14849792608693868, 'epoch': 3.0})

In [29]:
trainer.evaluate(tokenized_ds["test"])

100%|██████████| 298/298 [00:46<00:00,  6.47it/s]


{'eval_loss': 0.16764046251773834,
 'eval_accuracy': 0.9546409071818563,
 'eval_runtime': 47.6348,
 'eval_samples_per_second': 49.984,
 'eval_steps_per_second': 6.256,
 'epoch': 3.0}

In [38]:
import torch

torch.mean(tokenized_ds["test"]["labels"].float())

tensor(0.4998)

# LLM